Q: Write a decorator retry(n) that when applied to a function (e.g. a scraper) retries a failed function call up to n times.

In [10]:
import itertools
import requests

#Attempt on function which retries with delays
def retry(delays=(0, 1, 5, 30, 180, 600, 3600), # entering delay time, n = (0,1,5,30,180,600,3600)
#First retry of connection is immediate delay=0, then it will try after 1 sec, 5 sec etc.
          exception=Exception, #Adding exception
          report=lambda *args: None): 
    def wrapper(function):
        def wrapped(*args, **kwargs):
            problems = [] #Empty list with problems
            for delay in itertools.chain(delays, [ None ]):
                try:
                    return function(*args, **kwargs)
                except exception as problem: #exceptions are defined as problems?
                    problems.append(problem) #Appending problems to the empty list
                    if delay is None: #If a delay doesn't make a difference, stop and report the following
                        report("retryable failed definitely:", problems)
                        raise
                    else:
                        report("retryable failed:", problem,
                            "-- delaying for %ds" % delay)
                        time.sleep(delay)
        return wrapped
    return wrapper

In [11]:
#Sending a HTTP request to an internet server
def send_data(data):
  response = requests.post(URL, data=data)
  return response.content
#if you don't get an immediate response, the decorator function with
#delays can attempt after some time automatically.

In [12]:
def send_data(data):

  @retry()
  def send_post():
    return requests.post(URL, data=data)

  response = send_post()
  return response.content

In [13]:
@retry(delays=itertools.cycle([ 20 ]))

SyntaxError: unexpected EOF while parsing (<ipython-input-13-b725df7a2250>, line 1)

Inspired by "http://code.activestate.com/recipes/580745-retry-decorator-in-python/"